# __Age of Empires II aoe2recs.com API modules__

## __Define functions__
1. **Database** - Setup database connection strings and variables. Might have to change based on OS (Linux/Win)
2. **Fetch match details** - Call API and fetch match details

### __1. Database :__

In [50]:
def setup_mongo_conn():
    
    client = MongoClient('mongodb://localhost:27017/?readPreference=primary&appname=MongoDB%20Compass&ssl=false')    
    return(client)

def insert_match_mongo(match_data, db_conn):
    
    db_conn.aoe2.matches.insert_one(match_data) 
    
def fetch_latest_match_id_aoe2recs(db_conn):
    
    match_id = db_conn.aoe2.matches.find_one(sort=[("id", -1)])['id']
    return(match_id)

### __2. Fetch match details:__

In [45]:
def fetch_all_match_details_aoe2recs(match_id,query):
    
    query = query.replace('$$__$$',str(match_id))
    url = 'https://aoe2recs.com/api'
    res = requests.post(url, json={'query': query}).text
    res = json.loads(res)['data']['match']
    
    if res is None:
        res = {'id': -999}
        
    return(res)

In [54]:
def fetch_matches_iterate_aoe2recs(start,collect,db_conn):
    
    with open("/home/nemecys/Documents/GitHub/AgeofEmpires2_Voobly/Input/aoe2recs_match_data_query.txt", "r", encoding="utf-8") as text:
        query = text.read()
    
    for index in tqdm(range(collect),miniters = 1,bar_format='{desc:<5.5}{percentage:3.0f}%|{bar:60}{r_bar}'):
        
        # Fetch match details
        match_data = fetch_all_match_details_aoe2recs(start,query)
        
        # Write to mongoDB
        insert_match_mongo(match_data,db_conn)
        
        # Next iteration
        start+=1
    
    print('\nDONE !')
    print('\nAOE2 MATCHES DOWNLOADED : ' + str(collect))    